In [1]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import SimilarityMaps
from rdkit.Chem import RDConfig
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pgf import PdfPages
import os
from PIL import Image
import io
import numpy as np

RDKit WARNING: [22:11:27] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
plt.interactive('off')
mols = [m for m in Chem.SDMolSupplier(os.path.join(RDConfig.RDDocsDir, 'Book/data/cdk2.sdf'))]
for m in mols:
    AllChem.Compute2DCoords(m)
fps = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mols]

In [3]:
refmol = mols[0]
probmol = mols[1]
def simmap(refmol, probmol):
    im, score = SimilarityMaps.GetSimilarityMapForFingerprint(refmol, 
                                                               probmol, 
                                                               SimilarityMaps.GetMorganFingerprint, 
                                                               colorMap='coolwarm',
                                                               alpha=0.01,
                                                               size=(200,200))
    bio = io.BytesIO()
    im.savefig(bio, bbox_inches='tight', dpi=200)
    im = Image.open(bio)
    plt.clf()
    return im, score


In [4]:
im1, _ = simmap(mols[0], mols[0])
im2s, scores = [], []
for i, m in enumerate(mols[:5]):
    im2, score = simmap(mols[0], m)
    im2s.append(im2)
    scores.append(score)

In [5]:

plt.clf()
plt.interactive('off')
fig = plt.figure(figsize=(8.27, 11.69)) #A4 size
pdf_pages = PdfPages('report.pdf')


for i, m in enumerate(mols[:5]):
    ax1 = plt.subplot2grid((5,3),(i,0))
    im1 = im1.resize((400,400))
    ax1.imshow(im1, interpolation="catrom")
    ax1.get_xaxis().set_visible(False)
    ax1.get_yaxis().set_visible(False)   
    
    ax2 = plt.subplot2grid((5,3),(i,1))
    ax2.imshow(im2s[i], interpolation="catrom")
    ax2.get_xaxis().set_visible(False)
    ax2.get_yaxis().set_visible(False)
 
    ax3 = plt.subplot2grid((5,3),(i,2))
    ax3.table(cellText=[["SimScore",f"{scores[i]:.2}"]],bbox=(0,0,1,1), cellLoc="center")
    ax3.get_xaxis().set_visible(False)
    ax3.get_yaxis().set_visible(False)
    ax3.axis('off')
pdf_pages.savefig(fig, dpi=200)
pdf_pages.close()